In [1]:
#!pip install selenium
import re
import bs4
import time
import os
import pandas as pd
import numpy as np
import datetime
from selenium.webdriver.common.keys import Keys
from selenium import webdriver

In [40]:
my_driver = webdriver.Chrome('chromedriver.exe')

In [47]:
my_driver.get("https://hakwon.sen.go.kr/edusys.jsp?page=scs_m83000")

In [2]:
print(my_driver.current_url)

NameError: name 'my_driver' is not defined

In [49]:
my_gu = my_driver.find_element_by_css_selector("#selSearchZoneCode")
my_gu_list = my_gu.text.split()[:6]
print(my_gu_list)
n_gus = len(my_gu_list)

['강남구', '강동구', '강북구', '강서구', '관악구', '광진구']


In [50]:
my_gu = my_driver.find_element_by_css_selector("#selSearchZoneCode")
my_gu.send_keys(my_gu_list[0])
kind = my_driver.find_element_by_css_selector('#selSearchKindCode')
kind.send_keys("학교교과교습학원")
time.sleep(1)
typecode  = my_driver.find_element_by_css_selector('#selSearchTypeCode')
typecode.send_keys("독서실")
time.sleep(1)
search = my_driver.find_element_by_css_selector('#btn_search')
search.click()
time.sleep(1)
download = my_driver.find_element_by_css_selector('#btnExport')
download.click() 
time.sleep(1)

In [51]:
#나머지 페이지
i = 1 
while True :
    try :
        i += 1
        lst = my_driver.find_element_by_css_selector("#paglPg_page_" + str(i))
        lst.click()
        time.sleep(1)
        download = my_driver.find_element_by_css_selector('#btnExport').send_keys(Keys.ENTER)
        time.sleep(1)
    except : 
        break 
    if (i% 10) == 0 :
        nxt = my_driver.find_element_by_css_selector("#paglPg_next_btn")
        nxt.click()
        time.sleep(1)

In [52]:
for x in range(n_gus) :
    my_gu = my_driver.find_element_by_css_selector("#selSearchZoneCode")
    if (x != 0):
        my_gu.send_keys(Keys.DOWN)
        search = my_driver.find_element_by_css_selector('#btn_search').send_keys(Keys.ENTER)
        time.sleep(1)
        download = my_driver.find_element_by_css_selector('#btnExport').send_keys(Keys.ENTER)
        i = 1
        while True :
            try :
                i += 1
                lst = my_driver.find_element_by_css_selector("#paglPg_page_" + str(i))
                lst.click()
                time.sleep(1)
                download = my_driver.find_element_by_css_selector('#btnExport').send_keys(Keys.ENTER)
                time.sleep(1)
            except : 
                break 
            if (i% 10) == 0 :
                nxt = my_driver.find_element_by_css_selector("#paglPg_next_btn")
                nxt.click()
                time.sleep(1)

In [53]:
my_driver.close()

In [157]:
#엑셀 파일 이름 모으기 
os.chdir(r"C:\Users\pudding\Downloads")
file_list = os.listdir()
my_excel_files = []
for x in file_list :
    if x[-3:] == 'xls' and x[:5] == 'excel' :
        my_excel_files += [x] 

print(len(my_excel_files))

141


In [194]:
# 읽어 들여서 하나의 데이터 프레임으로 통합한다.
n_excels = len(my_excel_files)
for i in range(n_excels):  
    a_df = pd.read_excel(my_excel_files[i],header=4, na_values = [' ', '-'])
    if i == 0:
        df = a_df
    else:
        df = pd.concat([df,a_df], axis=0, ignore_index=True)

In [195]:
(df.isnull()).sum(axis=0)

순번                  0
학원명                 0
설립자\n강사             0
교습\n과정              0
교습\n과목              0
정원                  0
교습\n기간             12
총교습시간\n(분)         12
기숙사비               12
피복비                12
차량비                12
적용일\n(최종교습비변경일)     0
dtype: int64

In [198]:
df.dropna(axis=0, inplace=True)

In [201]:
df = df[df["적용일\n(최종교습비변경일)"]>'2018.01.01']
#df["기숙사비"] = df["기숙사비"].apply(lambda x: str(x).lstrip())
df["기숙사비"] = df["기숙사비"].str.lstrip()
#df.groupby("학원비")
#df.astype({"기숙사비":int})
#df[df.iloc[:,8] != 0]
df = df[df["기숙사비"] != '0'] 

In [202]:
df.to_excel(excel_writer='output.xls')